Along with my notes I've written up some mini exercises. If you are teacher you are more than welcome to use these question to influence questions you'd asked your class. Or copy these questions 1 for 1, I honestly don't mind. I'm more worried about making quality learning materail than preventing some inconsequential cheating.

# Chapter 2
Understanding the variety of sequences available in Python saves us from reinventingthe wheel, and their common interface inspires us to create APIs that properly supportand leverage existing and future sequence types.

A quick exercise

1. Group the following sequence types from the Python standard library as either a container sequence or flat sequence and explain the difference between each group.

    `bytes`, `str`, `list`, `deque`, `array`, `bytearray`, `memoryview`, `tuple`

    Note: `deque` comes from the `collections` library and `array` comes from the `array` library

    *Answers* (in any order)
    
    * **Container sequences:**
    
        1. `list`
        2. `tuple`
        3. `deque` or `collections.deque`
    
    * **Flat sequences:**
    
        1. `str`
        1. `bytes`
        1. `bytearray`
        1. `memoryview`
        1. `array` or `array.array`
    
    Explaination: *Container sequences hold references to the objects they contain, which may be of anytype, while flat sequences physically store the value of each item within its own memoryspace, and not as distinct objects. Thus, flat sequences are more compact, but they arelimited to holding primitive values like characters, bytes, and numbers.*

2. Regroup each of the above sequence types by its [*mutability*](https://en.wikipedia.org/wiki/Immutable_object). Explain the concept of mutability (and/or immuntabilty) and its usecase:

    *Answers* (in any order)

    * **Mutable sequences:**
    
        1. `list`
        1. `bytearray`
        1. `array` or `array.array`
        1. `deque` or `collections.deque`
        1. `memoryview`
    
    * **Immutable sequences:**
    
        1. `tuple`
        2. `str`
        3. `bytes`

3. (Extra question) Author Luciano states that buit-in concrete sequences do not actually subclass the `Sequence` and `MutableSequence` abstract base classes. Investigate and explain the reasoning for this. Follow up question. Why is the UML class diagram for `Sequence` and `MutableSequence` abstract base classes still useful for formalization of expected functionality for a full-featured sequence type?

    Figure out this answer on your own. Search online.


## List comprehension and readability

In short, if you ever have trouble understanding what is going on with list comprehension, the trick is to read the line out loud, 
<html>


<div class="jp-CodeMirrorEditor jp-Editor jp-InputArea-editor" data-type="inline">
     <div class="CodeMirror cm-s-jupyter">
<div class=" highlight hl-ipython3"><pre><span></span><span class="n">list_result</span> <span class="o">=</span> <span class="p">[</span><span class="n">do_something</span><span class="p">(</span><span class="n">x</span><span class="p">)</span> <span class="k">for</span> <span class="n">x</span> <span class="ow">in</span> <span class="n">some_list</span><span class="p">]</span>
</pre></div>
</div>
</div>

</html>

Task: 

1. Take the following code used to build a list of Unicode codepoints from a string and covert it to a list comprehension. Return the codes as a list of tuples with the symbol followed by its codepoint.

In [7]:
# code to convert
symbols = '$¢£¥€¤'
codes = []
for symbol in symbols:
    codes.append((symbol, ord(symbol)))

codes

[('$', 36), ('¢', 162), ('£', 163), ('¥', 165), ('€', 8364), ('¤', 164)]

In [8]:
# answer
symbols = '$¢£¥€¤'
codes = [ (symbol, ord(symbol)) for symbol in symbols]

codes

[('$', 36), ('¢', 162), ('£', 163), ('¥', 165), ('€', 8364), ('¤', 164)]

OR

In [10]:
codes = [ (symbol, ord(symbol)) for symbol in '$¢£¥€¤']
codes

[('$', 36), ('¢', 162), ('£', 163), ('¥', 165), ('€', 8364), ('¤', 164)]

----

## Listcomps have their own local scope, like functions. 

WARNING: This is not true in any version of Python 2.x

Below see how the valuable data stored in `x` is not screwed with by the list comprehension.

In [14]:
x = [1, 3, 5, 10,]

dummy = [x for x in 'ABC']
print('the x value is still:', x)
print('the dummy value is:  ', dummy)

the x value is still: [1, 3, 5, 10]
the dummy value is:   ['A', 'B', 'C']


This is significant as listcomps address several issues found in normal `for` loops:

1. Leaking values

    Regular ol `for` loop leak their values into the local name space. Consider the following

In [26]:
names = ['tim', 'tom', 'sally', 'sam']
lenths_of_names = []
for name in names:
    lenths_of_names.append(len(name))
# var name still exists and it still holds the item it was last assigned
print(lenths_of_names)
print(name) # will print'sam'

[3, 3, 5, 3]
sam


It turns out that `name` still exists after leaving the `for` loop. This is a variable leak. There are *some* cases where we could leverage variable leaks but most times it is a poor design choice.

Listcomps do not have this leak. If you tried to evaluate the variable declared in the list comprehension it will raise a `NameError` (assuming there isn't a variable with the same name in the relevent namespace)

Consider:

In [24]:
letters = ['a', 'b', 'c', 'd']
capitalized = [letter.capitalize() for letter in letters]
print(capitalized)
print(letter)

['A', 'B', 'C', 'D']


NameError: name 'letter' is not defined

I intentially did this to show that `NameError` would be raised. The string object `letter` is only local to the list comprehension namespace (the area within the `[]`). This is because Listcomps follow the same namespace rules as functions.

This also addresses the second issue of normal `for` loops:

2.  Masking variables in the surrounding namespace scope. 

consider the following in a normal `for` loop:
    

In [51]:
x = 'some important information'
some_other_info = [1,2,4,8,16,32]
plus1s = []
for x in some_other_info:
    plus1s.append(x + 1)
print(plus1s)
print(x)

[2, 3, 5, 9, 17, 33]
32


Looks like we lost `some important information`. That's not good. What happened here was the `x` in the for loop masked over the original x value. List comprehension doesn't have that problem.

In [52]:
x = 'some important information'
some_other_info = [1,2,4,8,16,32]
plus1s = [x+1 for x in some_other_info]
print(plus1)
print(x)

[2, 3, 5, 9, 17, 33]
some important information


Now you might be thinking that all of this could have been avoided if we were just more creative with our names. That's a good point, and you are correct, but what's more important is understanding exactly how python's sequence types work.

By the way, with the above knowledge you can write some truly odd syntax such as the following:


In [54]:
# Never show this to someone who is just getting started with python.
x = [1, 3, 5, 6]
triples = [x**3 for x in x]
print(triples)
print(x)

[1, 27, 125, 216]
[1, 3, 5, 6]


Did you know you can use list comprehensions in liue of map/filter?

In [55]:
symbols = '$¢£¥€¤'
# traditional approach
beyond_ascii = list(filter(lambda codepoint: codepoint > 127, map(ord,symbols)))
print(beyond_ascii)

# using list comprehension.
beyond_ascii = [ord(symbol) for symbol in symbols if ord(symbol) > 127]
print(beyond_ascii)

[162, 163, 165, 8364, 164]
[162, 163, 165, 8364, 164]


In this case, the list comprehension is faster than the traditional map/filter approach. And it's easier to read.

## Cartesian Products

This next feature of list comprehensions you may have also seen if you looked into the card deck class built in chapter 1.

In [56]:
# some setup
from collections import namedtuple
Card = namedtuple('Card', ['rank', 'suit'])
ranks = [str(n) for n in range(2, 11)] + list('JQKA')    
suits = 'spades diamonds clubs hearts'.split()

# the actual cartesian product
deck = [Card(rank, suit) for rank in ranks
                         for suit in suits]


Now you have a full deck to play with

# Generator Expressions

Generators save memory because they yield items 1 at a time, instead of building a whole list 

In [58]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
for tshirt in (f'{color} {size}' for color in colors for size in sizes):
    print(tshirt)

black S
black M
black L
white S
white M
white L


At no point was a list built. To clarify a bit more:

In [59]:
this_is_a_generator = (f'{color} {size}' for color in colors for size in sizes)
print(this_is_a_generator)

<generator object <genexpr> at 0x000001BEF9DF05F0>


Observe that what we built was a generator. We can now use it elsewhere.

In [60]:
for output in this_is_a_generator:
    print(output)

black S
black M
black L
white S
white M
white L


In [61]:
# we can check that the genertor is exhausted and will raise error StopIteration
next(this_is_a_generator)

StopIteration: 

------

# Tuples are not just immutable lists

But they can be useful as immutable lists (more on that in a bit)

## Tuples as Records

The ordering of items in a tuple can be (and often are) important. Each item in a tuple can be thought of as holding data for a *field*. In such cases the position of the items in the tuple give the tuple specific meaning (without the worry of the tuple be tampered with)

In [66]:
# Coordinates in latitude and longitude
ohara_airport = (41.9803, -87.9090) 

# youtuber's first and last name and their channel
youtuber = ('Tom', 'Scott', 'https://www.youtube.com/channel/UCBa659QWEk1AI4Tg--mrJ2A/videos') 

# a list of tuples which can then be sorted.
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'), ('ESP', 'XDA205856')] 


We can still perform slices on tuples, just like lists.

In [67]:
city = ('Tokyo', 2003, 32450, 0.66, 8014)
city[2]

32450


In [68]:
city[2:]

(32450, 0.66, 8014)

In [69]:
type(city[2:])

tuple

With the added advantage that the items inside of the tuples are "*safe*" from tampering.

In [79]:
# the following is suppose to raise errors
city = ('Tokyo', 2003, 32450, 0.66, 8014)
city[0] = 'Lima'

TypeError: 'tuple' object does not support item assignment

In [80]:
city.append('some new information')

AttributeError: 'tuple' object has no attribute 'append'

Some practice questions:

1. Consider the following list of tuples. Is it *safe* from tampering?

In [86]:
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'), ('ESP', 'XDA205856')] 

2. Consider the following tuple. Is it abosolutely safe from tampering? Test what happens when you attempt to add `city` to itself

In [88]:
city = ('Tokyo', 2003, 32450, 0.66, 8014)
city + city

Was this what you expected? Check the value of city now.

In [ ]:
city

What is going on here you might wonder? (I'll answer this later)

----

# Iterable unpacking (and tuple unpacking)

* **Basic unpacking: *parallel assignment***

    Each item in the sequence is assigned 1 for 1 to variables.

In [90]:
# list unpacking
mailbox = ['mail', 'spam']
something_important, something_useless = mailbox
print(something_important)
print(something_useless)

mail
spam


In [89]:
# tuples also have unpacking
ohara_airport = (41.9803, -87.9090) 
lattitude, longitude = ohara_airport
print(lattitude)
print(longitude)

41.9803
-87.909


We can use this parrallel assignment to elegantly swap variables without the need of a temporary variable (that's not say that lower level code doesn't use a temporary variable, just that we don't need to do that work ourselves)

In [93]:
y = 33
z = 10
y, z = z, y
print(y)
print(z)

10
33


Need to declare several variables on after another? You can set them up on the same line.

In [94]:
thing1, thing2, thing3, thing4 = ['hello', 'hi', 'yo', 'hey']

But, what happens when there is a missmatch between items to assign and variables to receive?

In [91]:
# more variables than items
foods = ['apple', 'bannana', 'carrot']
a, b, c, d = foods

ValueError: not enough values to unpack (expected 4, got 3)

In [92]:
# more items than variables
foods = ['apple', 'bannana', 'carrot', 'dragon fruit']
a, b, c = foods

ValueError: too many values to unpack (expected 3)

Both raise a `ValueError`. I've seen people use this fact to their advantage in try and except handlings. 

# Something Extra

Earlier I mentioned that there are some use-cases where it's necessary to access a leaked variable from a for loop and that doing so is considered poor practice.

Here's an example of just that.

In [34]:
# getting the first value that satisfys the "fizzbuzz" challenge given an arbitrary list:
values = [16, 25, 34, 3, 3, 5, 75, 100, 105]
has_fizzbuzz = False
for x in values:
    if x%3==0 and x%5==0:
        has_fizzbuzz = True
        break
if has_fizzbuzz:
    print('fizzbuzz')
    print('fizzbuzz value:', x)
else:
    print('no fizzbuzz')

fizzbuzz
fizzbuzz value: 75


The above is a terrible approach. It's excessive. It's just awful. I feel dirty just writting this example. Also this a classic example of why coding in `__main__` can be messy.

But, by using a function we can get the same results. In short we solved the problem without ever needing to resort to leveraging the leak. 

In [36]:
# this is much better

def get_fizzbizz_value(arr):
    for x in arr:
        if x%3==0 and x%5==0:
            print('fizzbuzz')
            print('fizzbuzz value:', x)
            return x
    print('no fizzbuzz')
    return -1

values = [16, 25, 34, 30, 3, 5,]
answer = get_fizzbizz_value(values)

fizzbuzz
fizzbuzz value: 30
